In [75]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

tree 모델보다 linear 모델이 outlier에 민감함


In [76]:
pd.options.display.max_columns = 99
pd.options.display.max_rows = 1000
#pd.set_option('display.max_rows', 1000)
train = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv.gz')
train

In [77]:
drop_index = train[train['GrLivArea'] > 4500].index
drop_index
train = train.drop(drop_index)

In [78]:
test = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv.gz')
test

In [79]:
train.corr()['SalePrice'].sort_values(ascending=False)

In [80]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16,8))
sns.scatterplot(train['GrLivArea'], train['SalePrice'])

In [81]:
import seaborn as sns

sns.displot(np.log(train['SalePrice']))

In [82]:
sns.displot(train['SalePrice'])

문제점
1. 정규분포가 아님
2. 이상치가 존재함 (70만, 80만)

In [83]:
all_data = pd.concat([train,test])

In [84]:
all_data_2 = all_data.drop(columns=['Id','SalePrice'])
all_data_2

선형모델은 column 개수가 많아도 학습 잘함

In [85]:
all_data_3 = pd.get_dummies(all_data_2)
all_data_3

In [86]:
all_data_3 = all_data_3.fillna(-1)

In [87]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
all_data_4 = std.fit_transform(all_data_3)
all_data_4

In [88]:
all_data_4 = pd.DataFrame(all_data_4, columns=all_data_3.columns)
all_data_4

In [89]:
train_2 = all_data_4[:len(train)]
test_2 = all_data_4[len(train):]

tree 모델은 따로 고려안해도 feature의 중요도를 고려함

linear 모델은 feature의 가중치 고려를 안함 -> 점수 안나옴

In [90]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha = 300)
ridge.fit(train_2, np.log(train['SalePrice']))

패턴 파악하기 !!

In [91]:
from sklearn.model_selection import cross_val_score

cvs = -cross_val_score(ridge, train_2, train['SalePrice'], cv=10, scoring='neg_mean_absolute_error')
cvs.mean()

In [92]:
for i in [10,20,30,50,70,100,200,300,500,700,1000]:
    ridge_cv = Ridge(alpha = i)
    cvs = -cross_val_score(ridge_cv, train_2, train['SalePrice'], cv=10, scoring='neg_mean_absolute_error')
    print(i, cvs.mean())

In [93]:
result = ridge.predict(test_2)
result

In [94]:
sub = pd.read_csv('/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz')
sub

In [95]:
sub['SalePrice'] = np.exp(result)
sub

In [96]:
sub.to_csv('sub.csv',index=0)